In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

def process_and_plot(nifti_file1, nifti_file2):
    """
    Loads 3D NIfTI files, processes them, and plots the result.

    Args:
        nifti_file1: Path to the first NIfTI file (3D).
        nifti_file2: Path to the second NIfTI file (4D).

    Returns:
        None
    """

    # Load the 3D NIfTI file
    img1 = nib.load(nifti_file1)
    data1 = img1.get_fdata()

    # Load the 4D NIfTI file
    img2 = nib.load(nifti_file2)
    data2 = img2.get_fdata()

    # Downsample the 3D image to match the dimensions of the 4D image
    data1_resampled = data1[:data2.shape[0], :data2.shape[1], :data2.shape[2]]

    # Create a binary mask from the 3D image
    mask = (data1_resampled > 0.1).astype(int)

    # Multiply the 4D image with the binary mask along the 4th dimension
    result = data2 * mask[:, :, :, np.newaxis]

    # Calculate the average along the first 3 dimensions
    avg_result = np.mean(result, axis=(0, 1, 2))

    # Plot the average result along the 4th dimension
    plt.plot(avg_result)
    plt.xlabel('Time')
    plt.ylabel('Average Value')
    plt.title('Average Value Over Time')
    plt.show()

# Replace with the actual paths to your NIfTI files
nifti_file1 = 'path/to/your/3d_nifti.nii.gz'
nifti_file2 = 'path/to/your/4d_nifti.nii.gz'

process_and_plot(nifti_file1, nifti_file2)

In [ ]:
import numpy as np
from scipy.io.wavfile import write
from pydub import AudioSegment

def image_to_audio(image_signal, output_file='output.wav', sample_rate=44100, duration=5):
    """
    Converts a mean image signal to a simple audio signal.

    Args:
        image_signal: The mean image signal as a 2D NumPy array.
        output_file: The desired output filename.
        sample_rate: The desired sample rate of the audio.
        duration: The desired duration of the audio in seconds.

    Returns:
        None
    """

    # Normalize the image signal
    normalized_signal = (image_signal - np.min(image_signal)) / (np.max(image_signal) - np.min(image_signal))

    # Reshape the signal into a 1D array
    signal_1d = normalized_signal.flatten()

    # Create an audio array with the desired sample rate and duration
    audio_array = np.interp(np.arange(0, duration * sample_rate), np.linspace(0, len(signal_1d) - 1, duration * sample_rate), signal_1d))

    # Scale the audio array to the range of 16-bit signed integers
    audio_array = (audio_array * 32767).astype(np.int16)

    # Write the audio array to a WAV file
    write(output_file, sample_rate, audio_array)

    # Create an AudioSegment object for further processing (optional)
    audio_segment = AudioSegment.from_wav(output_file)

    # Apply audio effects or modifications to the audio_segment (optional)
    # ...

    # Save the modified audio segment to a new file (optional)
    # audio_segment.export(output_file, format="wav")